# Tune GPT2 to generate controlled sentiment reviews
> Optimise GPT2 to produce IMDB movie reviews with controlled sentiment using a BERT sentiment classifier for rewards.

**WARNING:** We often experienced loss spikes in this examples which caused model training to fail or slow down. There is a [GitHub issue](https://github.com/lvwerra/trl/issues/101) to track the issue.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2-ctrl-training-setup.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


The experiment setup is very similar to the positive sentiment notebook. However, in this notebook we fine-tune GPT2 (small) to generate **controlled** movie reviews based on the IMDB dataset. The model gets the target sentiment and 5 tokens from a real review and is tasked to produce continuations with the targeted sentiment. The reward for the continuations is calculated with the logits of a BERT sentiment classifier. That reward is then used for PPO training.

## Setup experiment

### Import dependencies

In [1]:
# !pip install trl==0.11.4

In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
import random
import torch
import wandb
import time
import os
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from random import choices
import matplotlib.pyplot as plt

tqdm.pandas()
import sys
sys.path.append('/root/autodl-tmp/commit_generative_reinforcement_learning')
# from datasets import load_dataset

from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

from trl import (
    PPOTrainer,
    PPOConfig,
    AutoModelForCausalLMWithValueHead,
    create_reference_model,
)
from trl import AutoModelForSeq2SeqLMWithValueHead
# from trl.experimental.ppo import PPOTrainer

### Configuration

In [4]:
commit_pipe_kwargs = {"top_k": None, "function_to_apply": "none"}

config = PPOConfig(
    model_name="./codet5-sft-commit",
    steps=51200,
    learning_rate=1.41e-5,
    remove_unused_columns=False,
    log_with="wandb",
    batch_size=32,
    mini_batch_size=32,
)

txt_in_len = 1000
txt_out_len = 128
seed = 1


/root/autodl-tmp/commit_generative_reinforcement_learning/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [5]:
CSV_PATH = "/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv"
DIFF_COL = "diffs"
LABEL_COL = "labels"   # 如果后面不用 control token，可以不用

You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://huggingface.co/papers/1909.08593). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [6]:
policy_name = config.model_name
tokenizer = AutoTokenizer.from_pretrained(policy_name)

# 1) policy / ref policy: Seq2Seq + value head
policy = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(policy_name)
ref_policy = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(policy_name)

In [7]:
# 确保 eos_token_id 是正数
print("eos_token_id:", tokenizer.eos_token_id)
print("pad_token_id:", tokenizer.pad_token_id)

eos_token_id: 2
pad_token_id: 0


### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 500 characters long and take the first 1000 characters of each comment. The first filter we apply to avoid comments that are less than `txt_in_len` token long and the second to avoid tokenizing way more text than we actually need.

In [8]:
# import pandas as pd
# =========================
# 1) 读 CSV
# =========================
df = pd.read_csv(CSV_PATH)
df[DIFF_COL] = df[DIFF_COL].fillna("")

full_dataset = Dataset.from_pandas(df, preserve_index=False)

# =========================
# 2) 划分 70 / 15 / 15
# =========================
first_split = full_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = first_split["train"]     # ✅ 只用这个
tmp_dataset = first_split["test"]

second_split = tmp_dataset.train_test_split(test_size=0.5, seed=42)
valid_dataset = second_split["test"]
test_dataset = second_split["train"]

ds_splits = DatasetDict({
    "train": train_dataset,
    "valid": valid_dataset,
    "test":  test_dataset,
})

# =========================
# 3) 构造 prompt（与你原逻辑一致）
# =========================
def build_query(diff, max_chars=500):
    return (
        "Please read the following code and write comments in natural language:\n"
        + diff[:max_chars]
        + "\nComments:"
    )

# =========================
# 4) 只处理 train split
# =========================
train_ds = ds_splits["train"]

train_ds = train_ds.map(
    lambda x: {
        "input_ids": tokenizer.encode(
            build_query(x[DIFF_COL]),
            truncation=True,
            max_length=txt_in_len,
        )
    },
    batched=False,
)

train_ds = train_ds.map(
    lambda x: {
        "query": tokenizer.decode(x["input_ids"], skip_special_tokens=False)
    },
    batched=False,
)

# =========================
# 5) 设置 PyTorch 格式
# =========================
train_ds.set_format(
    type="torch",
    columns=["input_ids", "query"],  # ✅ 加上 query
)

print(train_ds)
print(train_ds[0])

Map:   0%|          | 0/1246 [00:00<?, ? examples/s]

Map:   0%|          | 0/1246 [00:00<?, ? examples/s]

Dataset({
    features: ['user', 'repo', 'commit', 'labels', 'msgs', 'diffs', 'feature', 'input_ids', 'query'],
    num_rows: 1246
})
{'input_ids': tensor([    1,  8496,   855,   326,  3751,   981,   471,  1045,  5678,   316,
        15145,  2653,    30,   203,  5413,  1493,  6845,   279,    19,  4816,
           19,  5254,    19,  6290,    19,  3341,    19,    70, 29017,  7277,
           19,  4327,  3567,    19,  4327,    19,  2159,    19,  7025,    19,
         2276,    18,  6290,   324,    19,  4816,    19,  5254,    19,  6290,
           19,  3341,    19,    70, 29017,  7277,    19,  4327,  3567,    19,
         4327,    19,  2159,    19,  7025,    19,  2276,    18,  6290,   203,
         2704,   585,  1965,  2130, 22087,   203,  1615,   374, 17877,   838,
           72,  3461,  3657,  9452,    71,   203,  6062,   342,  5206,    19,
         2011,   203,  9904,    15,   324,    19,  4816,    19,  5254,    19,
         6290,    19,  3341,    19,    70, 29017,  7277,    19,  4327,  

In [9]:
train_ds['query']

Column(['<s>Please read the following code and write comments in natural language:\ndiff --git a/src/main/java/org/buddycloud/channelserver/channel/node/configuration/Helper.java b/src/main/java/org/buddycloud/channelserver/channel/node/configuration/Helper.java\nnew file mode 100644\nindex 00000000..d141942c\n--- /dev/null\n+++ b/src/main/java/org/buddycloud/channelserver/channel/node/configuration/Helper.java\n@@ -0,0 +1,15 @@\n+package org.buddycloud.channelserver.channel.node.configuration;\n+\n+import java.util.Map;\n+\n+import org.xmpp.packet.IQ;\n+\n+public class Helper\n+{\n+\tpublic \nComments:</s>', '<s>Please read the following code and write comments in natural language:\ndiff --git a/src/libvaladoc/importer/girdocumentationimporter.vala b/src/libvaladoc/importer/girdocumentationimporter.vala\nindex 4186c27..9984dd5 100644\n--- a/src/libvaladoc/importer/girdocumentationimporter.vala\n+++ b/src/libvaladoc/importer/girdocumentationimporter.vala\n@@ -511,8 +511,11 @@ public cl

### Tokenize IMDB reviews

We tokenize all IMDB in advance to avoid tokenizing twice. In the first step we encode the queries and slice the first `txt_in_len` tokens. In a second step we decode these tokens back to text for later display.

In [10]:
# dataset[3]["input_ids"]

In [11]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [12]:
ppo_trainer = PPOTrainer(
    config, policy, ref_policy, tokenizer, train_ds, data_collator=collator
)

/root/autodl-tmp/commit_generative_reinforcement_learning/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.

KeyboardInterrupt



### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
else:
    device = ppo_trainer.accelerator.device
commit_pipe = pipeline(
    "text-classification", "/root/autodl-tmp/CommitFit/notebooks/E-3-best(70%)/my_awesome_model/checkpoint-390", device=device
)

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [ ]:
text = "Change hasException to hasThrowable--"
output = commit_pipe(text, **commit_pipe_kwargs)
output

In [ ]:
text = "Trying to extend the Scheduler interface according- to the comments at -19.--"
output = commit_pipe(text, **commit_pipe_kwargs)
output

In [ ]:
text = "RunAsync method for outputting multiple values--"
output = commit_pipe(text, **commit_pipe_kwargs)
output

The resulting reward signal:

In [ ]:
def extract_pipe_output(outputs):
    logits_dicts = []
    for out in outputs:    # 每个样本
        logits = {element["label"]: element["score"] for element in out}
        logits_dicts.append(logits)
    return logits_dicts

In [ ]:
output[1]["score"]

### Control token dict
We will append the control token at the beginning of each query to signal the model what the target sentiment is. Each control sequence consists of three tokens:

In [ ]:
# import torch
# 定义控制标签
ctrl_str = ["[Adaptive]", "[Perfective]", "[Corrective]"]

# 自动选择设备（GPU 优先）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 把 control string 编码成 token，用于 prepend 到输入
ctrl_tokens = {
    s: tokenizer.encode(s, return_tensors="pt").squeeze().to(device)
    for s in ctrl_str
}


In [ ]:
ctrl_tokens

### Reward function

In [ ]:
label_map = {
    "[Adaptive]": "LABEL_0",
    "[Perfective]": "LABEL_1",
    "[Corrective]": "LABEL_2"
}

In [ ]:
def language_quality_reward(text: str) -> float:
    """
    Heuristic language quality reward.
    Returns a scalar float (can be negative).
    """
    if text is None:
        return -0.5
    text = text.strip()
    if len(text) == 0:
        return -0.5

    # 1) 长度约束（过短通常是碎片）
    if len(text.split()) < 5:
        return -0.3

    # 2) 禁止 URL / License / Copyright 等噪声
    blacklist = ["http", "www", "license", "copyright", "©"]
    low = text.lower()
    if any(b in low for b in blacklist):
        return -0.5

    # 3) 简单句子结构奖励（首字母大写）
    if text[0].isupper():
        return 0.2

    return 0.0


def task_logit_to_reward(
    logits,
    tasks,
    responses=None,
    alpha: float = 1.0,   # 分类对齐 reward 权重
    beta: float = 1.0,    # 语言质量 reward 权重
):
    """
    Combine classifier-alignment reward and language-quality reward.

    Args:
        logits: List[Dict]  每个样本分类输出，如 {"LABEL_0":0.1,"LABEL_1":0.7,...}
        tasks:  List[str]   control token list，如 ['[Adaptive]', ...]
        responses: List[str] 生成的文本（用于语言质量 reward）。可为 None（则只用分类 reward）
        alpha: 分类 reward 权重
        beta:  语言质量 reward 权重

    Returns:
        rewards: List[torch.Tensor] 每个样本一个标量 reward tensor
    """
    rewards = []
    use_lang = responses is not None

    for i, (logit, task) in enumerate(zip(logits, tasks)):
        mapped_task = label_map[task]  # e.g. "[Adaptive]" -> "LABEL_0"
        if mapped_task not in logit:
            raise ValueError(
                f"Unknown task {mapped_task}, must be one of {list(logit.keys())}"
            )

        cls_r = float(logit[mapped_task])

        lang_r = 0.0
        if use_lang:
            lang_r = float(language_quality_reward(responses[i]))

        final_r = alpha * cls_r + beta * lang_r
        rewards.append(torch.tensor(final_r))

    return rewards


The following examples show the rewards for the cases where the classifier logit is 4, -4 and 0 for the three targets ['Adaptive','Perfective','Corrective']. The scaling is not perfect as it differs between neutral and the other two classes. This is something to further investigate in the future. Ideally, one would use the logit output for each class individually, but since there is no dedicated class for neutral this is a workaround.

In [ ]:
print(ctrl_str)

### Generation settings

In [ ]:
generation_kwargs = {
    "min_length": 1,   # -1 会报错，改成 1 或 0
    "top_k": 0,        # 设为 0 表示不启用 top-k
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": txt_out_len,
    "eos_token_id": tokenizer.eos_token_id,
}

## Optimize model

**Steps**

The training loop consists of the following steps:
1. Get a batch of queries and create random controls
2. Get the query responses from the policy
3. Join query and responses and tokenize for BERT analysis
4. Get sentiments for query/responses from BERT
5. Optimize policy with PPO using the (query, response, reward) triplet
6. Log all the training statistics

**Training time**

This step takes **~2h** on a P6000 GPU with the above specified settings.

In [ ]:
TASK2LABEL = {
    "[Corrective]": "LABEL_2",
    "[SomeTask]": "LABEL_1",
    "[Another]": "LABEL_0",
}

LABEL2ID = {"LABEL_0": 0, "LABEL_1": 1, "LABEL_2": 2}

In [ ]:
for epoch in range(10):
    for batch in tqdm(ppo_trainer.dataloader):
        (
            logs,
            game_data,
        ) = (
            dict(),
            dict(),
        )

        #### prepend a random control token
        task_list = choices(ctrl_str, k=config.batch_size)
        # print(task_list)
        game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]
        query_tensors = [
            torch.cat((ctrl_tokens[t], input_ids))
            for t, input_ids in zip(task_list, batch["input_ids"])
        ]

        #### get response from gpt2
        response_tensors = []
        for query in query_tensors:
            response = ppo_trainer.generate(query, **generation_kwargs)
            response_tensors.append(response.squeeze())
            # print(response)
        game_data["response"] = [
            tokenizer.decode(r.squeeze(), skip_special_tokens=True, clean_up_tokenization_spaces=True,) for r in response_tensors
        ]
        
        #### commit analysis
        texts = [r for q, r in zip(batch["query"], game_data["response"])]
        # print(texts)
        logits = extract_pipe_output(commit_pipe(texts, **commit_pipe_kwargs))
        # print(task_list)
        rewards = task_logit_to_reward(
            logits,
            task_list,
            responses=game_data["response"],  # ⭐ 用生成文本算语言质量 reward
            alpha=1.0,  # 分类对齐
            beta=1.0,   # 语言质量（建议先从 0.5~1.0 试）
        )
        # print(rewards)
        #### Run PPO training
        t = time.time()
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        # print(stats)
        for cs in ctrl_str:
            key = "env/reward_" + cs.strip("[]")
            stats[key] = np.mean(
                [r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs]
            )
        ppo_trainer.log_stats(stats, game_data, rewards)

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the following:

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2-ctrl-training-stats.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

## Model inspection

### Reward distribution
First, we can have a look at the reward distribution. Both the negative and positive rewards are clearly shifted to high rewards. The neutral rewards, however, are still centered around zero. There are a few possible explanations for this. There could be a bug in the code and the way the neutral rewards are calculated. Another problem could be that sentence sometimes start with a strong sentiment and it is hard for the model shift the sentiment towards neutral.

In [ ]:
# stats

In [ ]:
for ctrl_s in ctrl_str:
    vals = [
        r.item()
        for r, t in zip(rewards, task_list)
        if t == ctrl_s
    ]
    if len(vals) > 0:
        plt.hist(
            vals,
            bins=20,
            density=True,
            alpha=0.5,
            label=ctrl_s,
        )

plt.legend(loc="best")
plt.title("Reward Distribution (per control token)")
plt.xlabel("Reward")
plt.ylabel("Density")
plt.grid(True)
plt.show()

## Save model
Finally, we save the model to disk for later usage.

In [ ]:
policy.save_pretrained("codet5-msgs-ctrl")
tokenizer.save_pretrained("codet5-msgs-ctrl")

In [ ]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent=f"{'成功'}"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="PPO学习模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("强化学习模型训练完毕")